Notebook used to parse the raw reports generated with SPM and build the data matrices with hypometabolism data

In [1]:
import os
import re
import pandas as pd
import numpy as np
import pingouin
from datetime import datetime
from scipy.spatial.distance import hamming
from scipy.stats import chi2_contingency
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import BayesianRidge

In [ ]:
INPUT_REPORTS = {
    'AD'   : os.path.join('..', 'Data', 'DatasetforValidationReport_FDGPET_AD'),
    'bvFTD': os.path.join('..', 'Data', 'DatasetforValidationReport_FDGPET_FTD')
}
COGNITIVE_DATA_FILE = os.path.join('..', 'Data', 'DatasetforValidation.csv')

AAL_ROIS = [
 'precentral_l', 'precentral_r', 'postcentral_l', 'postcentral_r', 'rolandic_oper_l', 'rolandic_oper_r', 
    'frontal_sup_l', 'frontal_sup_r', 'frontal_mid_l', 'frontal_mid_r', 'frontal_inf_oper_l', 'frontal_inf_oper_r', 
    'frontal_inf_tri_l', 'frontal_inf_tri_r', 'frontal_sup_medial_l', 'frontal_sup_medial_r', 'supp_motor_area_l', 
    'supp_motor_area_r', 'paracentral_lobule_l', 'paracentral_lobule_r', 'frontal_sup_orb_l', 'frontal_sup_orb_r', 
    'frontal_med_orb_l', 'frontal_med_orb_r', 'frontal_mid_orb_l', 'frontal_mid_orb_r', 'frontal_inf_orb_l', 
    'frontal_inf_orb_r', 'rectus_l', 'rectus_r', 'olfactory_l', 'olfactory_r', 'temporal_sup_l', 'temporal_sup_r', 
    'heschl_l', 'heschl_r', 'temporal_mid_l', 'temporal_mid_r', 'temporal_inf_l', 'temporal_inf_r', 'parietal_sup_l', 
    'parietal_sup_r', 'parietal_inf_l', 'parietal_inf_r', 'angular_l', 'angular_r', 'supramarginal_l', 'supramarginal_r', 
    'precuneus_l', 'precuneus_r', 'occipital_sup_l', 'occipital_sup_r', 'occipital_mid_l', 'occipital_mid_r', 
    'occipital_inf_l', 'occipital_inf_r', 'cuneus_l', 'cuneus_r', 'calcarine_l', 'calcarine_r', 'lingual_l', 
    'lingual_r', 'fusiform_l', 'fusiform_r', 'temporal_pole_sup_l', 'temporal_pole_sup_r', 'temporal_pole_mid_l', 
    'temporal_pole_mid_r', 'cingulum_ant_l', 'cingulum_ant_r', 'cingulum_mid_l', 'cingulum_mid_r', 'cingulum_post_l', 
    'cingulum_post_r', 'hippocampus_l', 'hippocampus_r', 'parahippocampal_l', 'parahippocampal_r', 'insula_l', 'insula_r', 
    'amygdala_l', 'amygdala_r', 'caudate_l', 'caudate_r', 'putamen_l', 'putamen_r', 'pallidum_l', 'pallidum_r', 
    'thalamus_l', 'thalamus_r'
]

EXCLUDE_ROIS = [
    'cerebelum_4_5_r', 'cerebelum_3_r', 'cerebelum_6_r', 'cerebelum_6_r', 'cerebelum_crus2_r', 'cerebelum_crus2_l',
    'cerebelum_crus1_l', 'cerebelum_crus1_r', 'cerebelum_10_l', 'cerebelum_4_5_l', 'cerebelum_9_l', 'cerebelum_8_r', 
    'cerebelum_7b_l', 'cerebelum_10_r', 'cerebelum_6_l', 'cerebelum_8_l', 'cerebelum_7b_r', 'vermis_6',
    'cerebelum_3_l'
]

# Dictionary with the cognitive variables used and nomenclature in English
COGNITIVE_TEST = {
    # digit span forward and backward
    "Spandirecto": "Digit span forward",
    "Spanindirecto": "Digit span backward",
    
    # Corsi’s test forward and backward,
    "Corsidirecto": "Corsi’s test forward",                 
    "Corsiinverso": "Corsi’s test backward",    
    
    # Trail Making Test part A and B
    "TMTa": "TMT A",
    "TMTb": "TMT B",
    
    # Symbol Digit Modalities Test
    "SDMT": "SDMT",
    
    # Stroop Color-Word Interference test
    "Stroop1": "SCWIT word reading",
    "Stroop2": "SCWIT color naming",
    "Stroop3": "SCWIT interference",
    
    # Tower of London-Drexel version
    "ToLtotalcorrectos": "ToL correct",
    "ToLmovtotales"    : "ToL movements",
    "ToLinicio"        : "ToL start",
    "ToLejecucion"     : "ToL execution",
    "ToLresolucion"    : "ToL resolution",
    
    # Boston Naming Test
    "BostonNT": "BNT",
    
    # Semantic fluency (animals)
    "Fluenciaanimalesescalar": "Semantic fluency", 
    
    # Letter fluency (words beginning with “p”),
    "FluenciaPescalar": "Letter fluency",
    
    # Free and Cued Selective Reminding Test
    "FCSRTL1"      : "FCSRT free recall 1",                           
    "FCSRTLT"      : "FCSRT total free recall",                           
    "FCSRTtotal"   : "FCSRT total recall",                     
    "FCSRTdifLibre": "FCSRT delayed free recall",               
    "FCSRTdifTOTAL": "FCSRT delayed total recall",       
    
    # Rey-Osterrieth Complex Figure (copy accuracy, copy time, memory at 3 and 30 minutes, and recognition memory)
    "ReyCopia"         : "ROCF copy accuracy",
    "Rey3min"          : "ROCF memory 3 min",
    "Rey30min"         : "ROCF memory 30 min" ,
    "ReyTiempo"        : "ROCF time",
    "ReyReconocimiento": "ROCF recognition",
    
    # Benton’s Judgment Line Orientation,
    "JLOescalar": "JLO",
    
    # Visual Object and Space Perception Battery (object decision, progressive silhouettes, 
    # discrimination of position, and number location
    "VOSPdecision"     : "VOSP object decision",
    "VOSPsiluetas"     : "VOSP progressive silhouettes",
    "VOSPdiscriposi"   : "VOSP discrimination of position",
    "VOSPlocaliznumero": "VOSP number location",
    

}

# Dictionary with the demographic variables
DEMO_DATA = {
    'Sexo': 'sex',
    'Edadactual': 'age',
    'AñosEducacion': 'years_of_formal_education'
}

# Cognitive test processing

In [3]:
# load and process cognitive data (same pipeline as with the training data)
cognitive_data = pd.read_csv(COGNITIVE_DATA_FILE, sep=';') 

unique_pet_ids = cognitive_data['CD_PET'].astype(int).values.tolist()

assert len(set(unique_pet_ids)) == len(unique_pet_ids), 'data contains duplicated'

cognitive_data = cognitive_data.set_index('CD_PET')[    
    list(DEMO_DATA.keys()) +
    list(COGNITIVE_TEST.keys())
].rename(columns={
    **DEMO_DATA,
    **COGNITIVE_TEST,
}, errors='raise').copy()

# convert tests to float
for cog_test in COGNITIVE_TEST.values():
    cognitive_data[cog_test] = cognitive_data[cog_test].apply(lambda v: float(v) if v != ' ' else np.nan)

# convert demographic variables to float
cognitive_data['age'] = cognitive_data['age'].astype(float)
cognitive_data['years_of_formal_education'] = cognitive_data['years_of_formal_education'].astype(float)

cognitive_data.loc[cognitive_data["SCWIT color naming"] > 20, 'SCWIT color naming'] = np.nan
cognitive_data.loc[cognitive_data["SCWIT interference"] < 2.0, "SCWIT interference"] = np.nan
cognitive_data.loc[cognitive_data["BNT"] > 20, 'BNT'] = np.nan

# for patients with dementia imputate the missing values in ToL and Rey
# using the lower values as these patients didn't complete the test because
# they cognitive impairment
variables_floor_imputation = [
    "ToL correct",
    "ToL movements",
    "ToL start",
    "ToL execution",
    "ToL resolution",
    "ROCF copy accuracy",
    "ROCF memory 3 min",
    "ROCF memory 30 min" ,
    "ROCF time",
    "ROCF recognition",
]

for var in variables_floor_imputation:
    cognitive_data.loc[cognitive_data[var].isna(), var] = 2.0

# Add FDG-PET information

In [4]:
final_df = []
verbose = False
for key, file in INPUT_REPORTS.items():
    # read file content
    with open(file) as f:
        file_content = f.read()
    
    # extract information
    pet_ids = {}
    valid_line = False
    for line in file_content.split('\n'):

        # search for PET ids
        pet_id_match = re.search(r'/(\d+)/', line)

        # line associated with a PET id
        if pet_id_match:
            cd_pet = int(pet_id_match.group(1))

            # check if the extracted id is the cognitive data ids
            if cd_pet not in unique_pet_ids:
                if verbose:
                    print('ERROR. CD_PET "{}" not found'.format(cd_pet))
                valid_line = False
            else:
                pet_ids[cd_pet] = {
                    roi: 0 for roi in AAL_ROIS
                }
                valid_line = True
                if verbose:
                    print('Success match for CD_PET "{}"'.format(cd_pet))

        else:
            if not valid_line:   # missing cd_pet
                continue

            # search for number of hypometabolic voxels
            roi_hypo_match = re.match(r'^\d.*', line)

            # there have been a match
            if roi_hypo_match:
                roi_match = None

                # process AAL ROIs
                if '(aal)' in line:
                    line = line.replace('(aal)', '')
                    roi_match = re.match(r'(\d+)\s+(.*?)\s+', line)

                ## process Brodman ROIs (omitted)
                #if 'brodmann' in line:
                #    line = line.replace(' area ', '_')
                #    roi_match = re.match(r'(\d+)\s+(.*)', line)

                if roi_match:

                    # extract ROI label and number of hypometabolic voxels
                    nvoxels, roi = int(roi_match.group(1)), roi_match.group(2).lower()

                    # omit ROI
                    if roi in EXCLUDE_ROIS:
                        continue

                    # save ROI information   
                    pet_ids[cd_pet][roi] = nvoxels

    pet_df = pd.DataFrame(pet_ids).T
    pet_df['diagnosis'] = key
    final_df.append(pet_df)

final_df = pd.concat(final_df, axis=0)
final_df.index.names = ['CD_PET']

In [5]:
# merge cognitive data and FDG-PET data 
df = cognitive_data.join(final_df, how='inner')
assert not df.index.duplicated().any()

# add diagnosis information to match train data structure 
df['diagnosis_AD'] = 0
df['diagnosis_HC'] = 0
df['diagnosis_bvFTD'] = 0

df.loc[df.diagnosis == 'bvFTD', 'diagnosis_bvFTD'] = 1
df.loc[df.diagnosis == 'AD', 'diagnosis_AD'] = 1

# Analyze missing values profile

In [6]:
missing_profile = pd.concat([
    df.isna().sum(),
    (df.isna().sum() / df.shape[0]) * 100
], axis=1)
missing_profile.columns = ['n_missing', 'perc_missing']
missing_profile = missing_profile.loc[missing_profile['n_missing'] > 0].copy().sort_values(by='n_missing', ascending=False)
missing_profile

,n_missing,perc_missing
JLO,8,20.0
FCSRT total free recall,3,7.5
FCSRT delayed total recall,3,7.5
FCSRT delayed free recall,3,7.5
FCSRT total recall,3,7.5
BNT,3,7.5
FCSRT free recall 1,3,7.5
TMT B,2,5.0
VOSP number location,2,5.0
SCWIT interference,1,2.5


In [7]:
subject_miss_perc = pd.DataFrame(
    df[list(COGNITIVE_TEST.values())].isna().sum(axis=1) / len(COGNITIVE_TEST) * 100, 
    columns=['perc'])

In [8]:
# perform imputation based on training data
train_data = pd.read_parquet(os.path.join('..', 'data', 'final_data_MICE.parquet'))

In [9]:
# create a copy of the data to be imputed
train_data_mice = train_data[
    list(COGNITIVE_TEST.values()) + 
    list(DEMO_DATA.values())].copy()
test_data_mice = df[
    list(COGNITIVE_TEST.values()) + 
    list(DEMO_DATA.values())].copy()

# create imputer (same configuration than the one used by Pedro)
mice_imputer = IterativeImputer(
    estimator=BayesianRidge(), 
    max_iter=100,
    random_state=1997
)

# standarize the data
scaler = StandardScaler()
scaler = scaler.fit(train_data_mice)
train_data_mice_std = scaler.transform(train_data_mice)
test_data_mice_std = scaler.transform(test_data_mice)

# fit estimator
mice_imputer_fitted = mice_imputer.fit(train_data_mice_std)

test_data_mice_imp = mice_imputer_fitted.transform(test_data_mice_std)

# de-standarize the variables
test_data_mice_imp = scaler.inverse_transform(test_data_mice_imp)
test_data_mice_imp = pd.DataFrame(
    test_data_mice_imp, 
    columns=test_data_mice.columns,
    index=test_data_mice.index)

# Data exportation

In [10]:
# format the final dataframe
df_test_final_imp = test_data_mice_imp.join(
    df[[c for c in df.columns if c not in test_data_mice_imp.columns]],
    how='inner')

# export the data
# df_test_final_imp_subsample.to_parquet(os.path.join('..', 'data', 'final_data_MICE_val_cohort_v1.parquet'))
# df_test_final_imp_subsample.to_csv(os.path.join('..', 'data', 'final_data_MICE_val_cohort_v1.csv'))

# Compare brain hypometabolism between cohorts

In [11]:

for condition in ['bvFTD', 'AD']:
    cognition_comparison = []
    metabolism_comparison = []

    train_data_cond = train_data.loc[train_data.diagnosis == condition]
    test_data_cond = df_test_final_imp.loc[df_test_final_imp.diagnosis == condition]
    
    # compare cognitive tests
    for cog_test in COGNITIVE_TEST.values():
        cog_test_stat = pingouin.ttest(
            train_data_cond[cog_test].values,
            test_data_cond[cog_test].values
        )
        cog_test_stat['diagnosis'] = condition
        cog_test_stat['mean_train'] = train_data_cond[cog_test].mean()
        cog_test_stat['mean_valid'] = test_data_cond[cog_test].mean()
        cog_test_stat['std_train'] = train_data_cond[cog_test].std()
        cog_test_stat['std_valid'] = test_data_cond[cog_test].std()
        cog_test_stat.index = [cog_test]
        cog_test_stat.index.names = ['Test']
        cognition_comparison.append(cog_test_stat)
        
    for roi in AAL_ROIS:
        # calculate proportions
        train_data_prop30 = np.sum((train_data_cond[roi] >= 30)) / len(train_data_cond) * 100
        test_data_prop30 = np.sum((test_data_cond[roi] >= 30)) / len(test_data_cond) * 100
        train_data_prop100 = np.sum((train_data_cond[roi] >= 100)) / len(train_data_cond) * 100
        test_data_prop100 = np.sum((test_data_cond[roi] >= 100)) / len(test_data_cond) * 100
        
        if train_data_prop30 < 5 or train_data_prop100 < 5:
            continue

        # create contingency table
        contingency_table30 = np.array([
            [np.sum((train_data_cond[roi] < 30)), np.sum((train_data_cond[roi] >= 30))],
            [np.sum((test_data_cond[roi] < 30)), np.sum((test_data_cond[roi] >= 30))]])

        # perform chi2
        chi230, p_val30, _, _ = chi2_contingency(contingency_table30)

        # create contingency table
        contingency_table100 = np.array([
            [np.sum((train_data_cond[roi] < 100)), np.sum((train_data_cond[roi] >= 100))],
            [np.sum((test_data_cond[roi] < 100)), np.sum((test_data_cond[roi] >= 100))]])

        # perform chi2
        chi2100, p_val100, _, _ = chi2_contingency(contingency_table100)

            
        metabolism_comparison.append({
            'ROI': roi,

            'prop_train (th 30)': train_data_prop30,
            'prop_valid (th 30)': test_data_prop30,
            'diff propotion (th 30)': train_data_prop30 - test_data_prop30,
            'chi2 statistic (th 30)': chi230,
            'p-value (th 30)': p_val30,

            'prop_train (th 100)': train_data_prop100,
            'prop_valid (th 100)': test_data_prop100,
            'diff propotion (th 100)': train_data_prop100 - test_data_prop100,
            'chi2 statistic (th 100)': chi2100,
            'p-value (th 100)': p_val100,
        })
        
    # format dataframes
    cognition_comparison_df = pd.concat(cognition_comparison, axis=0)
    cognition_comparison_df = cognition_comparison_df.drop(columns=['dof', 'alternative', 'BF10'])

    metabolism_comparison_df = pd.DataFrame(metabolism_comparison)
    metabolism_comparison_df = metabolism_comparison_df.set_index('ROI')
    
    # export dataframes
    cognition_comparison_df.to_csv(
        os.path.join(
            '..', 'results', '%s_cognition_ttest_%s_v1.csv' % (datetime.now().strftime('%Y%m%d'), condition) 
        ))
    metabolism_comparison_df.to_csv(
        os.path.join(
            '..', 'results', '%s_metabolism_chi2_%s_v1.csv' % (datetime.now().strftime('%Y%m%d'), condition) 
        ))